This notebook goes over the ways to add custom span attributes.

In [ ]:
# Ensure before any TruLens packages are imported this environment variable is
# set.

import os

os.environ["TRULENS_OTEL_TRACING"] = "1"

In [ ]:
# Create an example RAG app. Note we have three different ways to add attributes
# to the span. They are described by the functions:
# 1. retrieve_contexts_with_function_signature_attributes
# 2. retrieve_contexts_with_arbitrary_attributes
# 3. retrieve_contexts_with_arbitrary_attributes_using_standard_otel

from typing import List

from opentelemetry import trace
from trulens.core.otel.instrument import instrument


class MyRAG:
    @instrument()
    def retrieve_contexts_with_no_custom_attributes(
        self, query: str
    ) -> List[str]:
        """This function has no custom attributes."""
        return ["context 1", "context 2"]

    @instrument(
        attributes={
            "custom_attr__query": "query",
            "custom_attr__results": "return",
        }
    )
    def retrieve_contexts_with_function_signature_attributes(
        self, query: str
    ) -> List[str]:
        """
        This function has attributes from the function signature. This works via
        passing in a dictionary to the `attributes` parameter of the
        `@instrument` decorator where the keys are function arguments or
        "return" for the return value.

        Note that generally this is unnecessary as the function signature is
        already logged as span attributes by default. Specifically the argument
        "x" will be logged as "ai.observability.call.kwargs.x" and the return
        value will be logged as "ai.observability.call.return".
        """
        return ["context 3", "context 4"]

    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            "custom_attr__uppercased_query": kwargs["query"].upper()
        }
    )
    def retrieve_contexts_with_arbitrary_attributes(
        self, query: str
    ) -> List[str]:
        """
        This function has arbitrary attributes computed via supplying to the
        `@instrument` decorator's `attributes` parameter a function that takes
        in:
        1. The return value.
        2. Exception thrown if any.
        3. Positional arguments.
        4. Keyword arguments (all positional arguments will also be here).
        And returns a dictionary describing the span attributes to set and their
        values.
        """
        return ["context 5", "context 6"]

    @instrument()
    def retrieve_contexts_with_arbitrary_attributes_using_standard_otel(
        self, query: str
    ) -> List[str]:
        """
        This function has arbitrary attributes added via during the function
        execution getting the current span and directly setting attributes to
        it.
        """
        trace.get_current_span().set_attribute(
            "custom_attr__repeated", query + query
        )
        return ["context 7", "context 8"]

    @instrument()
    def generate_response(self, query: str, contexts: List[str]) -> str:
        return "response"

    @instrument()
    def query(self, query: str) -> str:
        contexts = (
            self.retrieve_contexts_with_no_custom_attributes(query)
            + self.retrieve_contexts_with_function_signature_attributes(query)
            + self.retrieve_contexts_with_arbitrary_attributes(query)
            + self.retrieve_contexts_with_arbitrary_attributes_using_standard_otel(
                query
            )
        )
        return self.generate_response(query, contexts)

In [ ]:
# Create the TruSession and TruApp.

from trulens.apps.app import TruApp
from trulens.core import TruSession

tru_session = TruSession()  # By default, this will use a local SQLite database.
tru_session.reset_database()

app = MyRAG()
tru_app = TruApp(
    app=app,
    app_name="My App",
    app_version="1.0.0",
)

In [ ]:
# Invoke the app.

with tru_app:
    app.query("query")

In [ ]:
# View the spans in the database and their custom attributes.

import pandas as pd
import sqlalchemy as sa

tru_session.force_flush()  # This forces all spans to be flushed.

db = tru_session.connector.db
with db.session.begin() as db_session:
    q = sa.select(db.orm.Event).order_by(db.orm.Event.start_timestamp)
    df = pd.read_sql(q, db_session.bind)
df["name"] = df["record"].apply(lambda x: x["name"])
df["custom_attributes"] = df["record_attributes"].apply(
    lambda curr: {
        k: v for k, v in curr.items() if k.startswith("custom_attr__")
    }
)
df = df[["name", "custom_attributes"]]
for _, row in df.iterrows():
    print(row["name"])
    print("\t" + str(row["custom_attributes"]))
    print()